In [1]:
def process_log_file(file_path):
    events = []
    current_event = []

    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # 行が空でない場合
                current_event.append(line)
            else:
                if current_event:  # 現在のセクションが空でない場合
                    events.append(''.join(current_event))  # 行を結合してセクションを追加
                    current_event = []

        # 最後のセクションを追加（空行で区切られない場合も考慮）
        if current_event:
            events.append(''.join(current_event))

    return events

def filter_events_by_search_str(strings, search_strings):
    """
    指定された検索文字列のいずれかを含む文字列だけを抽出する関数。

    :param strings: 検索対象の文字列のリスト
    :param search_strings: 検索する文字列のリスト
    :return: 検索文字列のいずれかを含む文字列のリスト
    """
    filtered = []
    for s in strings:
        if any(search in s for search in search_strings):
            filtered.append(s)
    return filtered

def write_events_to_file(sections, output_path):
    with open(output_path, 'w') as file:
        for section in sections:
            file.write(section + '\n')  # セクションの後に空白行を追加


events = process_log_file("./results/General-#0.elog")
search_strings = [
    "DEBUG:Sending (inet::physicallayer::RadioFrame)GeoNet packet from (artery::VanetRadio)radio",
    "DEBUG:Computing whether reception is possible"
]
filtered_events = filter_events_by_search_str(events, search_strings)
write_events_to_file(filtered_events, "./results/filtered-General-#0.elog")






In [1]:
import json
import os
from collections import defaultdict

def process_log_file(file_path):
    events = []
    current_event = []

    with open(file_path, 'r') as file:
        for line in file:
            if line.strip():  # Check if the line is not empty
                current_event.append(line.strip())
            else:
                if current_event:
                    events.append(current_event)
                    current_event = []

        # Append the last event if it doesn't end with an empty line
        if current_event:
            events.append(current_event)

    return events


def extract_event_id(event):
    first_item = event[0]  # Get the first item from the list
    if "E #" in first_item:
        parts = first_item.split()
        for i, part in enumerate(parts):
            if part == "E":  # Check for the "E" token
                if i + 2 < len(parts):  # Ensure there's a number after "E #"
                    return parts[i + 2]  # Return the number following "E #"
    return None  # Return None if "E #" pattern is not found


def filter_events_by_strings(events):

    search_strings = ["KF"]

    filtered_events = [event for event in events if any(any(substring in line for substring in search_strings) for line in event)]
    return filtered_events

def extract_following_events_based_on_occurrences(events, search_string):
    results = {}
    for i, event in enumerate(events[:]):
        # Count occurrences of the search string in the current event
        count = sum(search_string in line for line in event)
        
        if count > 0:
            # Extract (count - 1) subsequent events
            count -= 1  # Adjust to extract one fewer than the count of occurrences
            buffer_log_num = 1 # tranmitterのログからすぐにreceiverのログに行かない可能性があるためバッファを設ける
            subsequent_events = events[i+1:i+1+count+buffer_log_num] if i+1+count <= len(events) else events[i+1:]

            results[extract_event_id(event)] = {}
            results[extract_event_id(event)]["transmitter_event"] = event
            results[extract_event_id(event)]["receiver_events"] = subsequent_events
    
    return results

def extract_transmitter_id_and_timestamp(log_entries):
    transmitter_id = None
    startTime = None
    endTime = None
    transmitter_position = None
    for entry in log_entries:
        if "transmitterId =" in entry:
            parts = entry.split(',')
            # Extract transmitterId and startTime
            for part in parts:
                if "transmitterId" in part:
                    transmitter_id = part.split('=')[-1].strip()
                if "startTime" in part:
                    startTime = part.split('=')[-1].strip()
                    startTime = float(startTime)
                if "endTime" in part:
                    endTime = part.split('=')[-1].strip()
                    endTime = float(endTime)
                
                
                # Attempt to extract position if available
                if "startPosition =" in entry:
                    # Correctly capture the full coordinates in the tuple format
                    transmitter_position = entry.split("startPosition =")[1].split('),')[0].strip() + ')'
    return transmitter_id, startTime, endTime, transmitter_position


# from multiple receiver logs, extract receiver_id and reception possibility
def extract_receiver_id_and_reception_possibility(log_entries_2list):
    results = {}
    receivable_id_list = []

    for log_entries in log_entries_2list:
        receiver_id = None
        position = None
        reception_status = "possible"
        startTime = None
        endTime = None

        for entry in log_entries:


            if "receiverId =" in entry:
                # Extract the receiver ID from the entry
                receiver_id = entry.split('receiverId =')[-1].split(',')[0].strip()
            # Attempt to extract position if available
            if "startPosition =" in entry:
                # Correctly capture the full coordinates in the tuple format
                position = entry.split("startPosition =")[1].split('),')[0].strip() + ')'
            # Update the reception status based on current entry details
            if "reception is impossible" in entry:
                reception_status = "impossible"


            # if "startTime" in entry:
            #     startTime = entry.split('=')[-1].strip()
            # if "endTime" in entry:
            #     endTime = entry.split('=')[-1].strip()

        if receiver_id is not None:
            results[receiver_id] = {
                "position": position,
                "reception_status": reception_status
            }

            if reception_status == "possible":
                receivable_id_list.append(int(receiver_id))
            
    # order by receiver_id
    results = dict(sorted(results.items(), key=lambda x: x[0]))
    receivable_id_list = sorted(set(receivable_id_list))
        
    return results, receivable_id_list


def extract_paramater_values(events_dict):
    for key, value in events_dict.items():
        transmitter_id, transmitter_startTime, transmitter_endTime,transmitter_position = extract_transmitter_id_and_timestamp(value["transmitter_event"])
        receive_results, receivable_id_list = extract_receiver_id_and_reception_possibility(value["receiver_events"])
        events_dict[key]["transmitter_id"] = transmitter_id
        events_dict[key]["startTime"] = transmitter_startTime
        events_dict[key]["endTime"] = transmitter_endTime
        events_dict[key]["transmitter_position"] = transmitter_position
        events_dict[key]["receiver_results"] = receive_results
        events_dict[key]["receivable_id_list"] = receivable_id_list
        events_dict[key]["receivable_id_count"] = len(receivable_id_list)

        # Remove the transmitter_event and receiver_events keys
        events_dict[key].pop("transmitter_event")
        events_dict[key].pop("receiver_events")
    
    return events_dict

def ensure_directory_exists(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

def split_json_by_transmitter_id(data):
    # transmitter_idでデータを分割するための辞書
    split_data = defaultdict(dict)

    # 各エントリをtransmitter_idごとに分割
    for key, entry in data.items():
        transmitter_id = entry.get('transmitter_id')
        if transmitter_id is not None:
            if transmitter_id not in split_data:
                split_data[transmitter_id] = {}
            split_data[transmitter_id][key] = entry

    # transmitter_idごとのデータを辞書に格納
    result = {id: entries for id, entries in split_data.items()}

    return result


        

def main():

    speed = 80 # km/hour
    events = process_log_file("./results/filtered-General-#0.elog")

    # # write event.json file
    # with open("./results/events.json", 'w') as f:
    #     json.dump(events[:1000], f, indent=4)

    log_folder_path = "./results/speed" + str(speed) + "/250vehicle/"
    ensure_directory_exists(log_folder_path)
    filtered_events_file_path = log_folder_path + "filtered_events.json"
    # extracted_events_file_path = log_folder_path + "/extracted_events.json"
    # print(events)

    filtered_string = "DEBUG:Sending (inet::physicallayer::RadioFrame)GeoNet packet"
    filtered_events = extract_following_events_based_on_occurrences(events, filtered_string)
  
    # # write to the json file
    # with open(filtered_events_file_path, 'w') as f:
    #     json.dump(filtered_events, f, indent=4)

    # Extract transmitterId, startTime, receiverId, and reception possibility
    extracted_events = extract_paramater_values(filtered_events)

    # if transmitter_id is not 0 , then the event is deleted form the dictionary
    split_events = split_json_by_transmitter_id(extracted_events)

    for transmitter_id, split_events_item in split_events.items():        
        # write to the json file even if the file is empty
        ensure_directory_exists(log_folder_path + str(transmitter_id) + "/")
        with open(log_folder_path + str(transmitter_id) + "/extracted_events.json", 'w') as f:
            json.dump(split_events_item, f, indent=4)



if __name__ == "__main__":
    main()



In [3]:
def ensure_directory_exists(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

log_folder_path = "./results/speed" + str(70) + "/300vehicle/"
ensure_directory_exists(log_folder_path)

ensure_directory_exists(log_folder_path + str(0) + "/")